In [1]:
from loader import load_classification_dataset
from simple_train import train_evaluate
from pretrain import pretrain_finetune_evaluate

import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
ds_path = os.path.join(os.getcwd(), 'datasets')

In [3]:
LSTM_UNITS = 16
LSTM_LAYERS = 1

MLP = [16, 8]

ACT = 'swish'

def print_results(r):
    train_result, eval_result = r
    print(f'Train | Loss: {train_result[0]:.2f} - Acc: {train_result[1]:.2f}')
    print(f'Eval  | Loss: {eval_result[0]:.2f} - Acc: {eval_result[1]:.2f}')

def print_accs(r):
    r = np.array(r)
    m = np.mean(r)
    std = np.std(r)
    print()
    print(f'Accuracy | Mean: {m:.2f} - Std: {std:.2f} | Max: {np.max(r):.2f}')

In [4]:
def lstm_trunk(x):
    for _ in range(LSTM_LAYERS-1):
        x = keras.layers.LSTM(LSTM_UNITS, return_sequences=True)(x)

    x = keras.layers.LSTM(LSTM_UNITS)(x)

    for d in MLP:
        x = keras.layers.Dense(d, activation=ACT)(x)

    return x

In [5]:
def lstm_autoencoder(input_x):
    x = input_x
    for _ in range(LSTM_LAYERS-1):
        x = keras.layers.LSTM(LSTM_UNITS, return_sequences=True)(x)

    x = keras.layers.LSTM(LSTM_UNITS, name='trunk')(input_x)

    x = keras.layers.RepeatVector(input_x.shape[1])(x)

    for _ in range(LSTM_LAYERS):
        x = keras.layers.LSTM(LSTM_UNITS, return_sequences=True)(x)

    _mlp_dim = input_x.shape[-1]
    for _ in range(len(MLP)-1):
        x = keras.layers.TimeDistributed(keras.layers.Dense(_mlp_dim, activation=ACT))(x)

    return keras.layers.TimeDistributed(keras.layers.Dense(input_x.shape[-1]))(x)

In [8]:
epochs=200
verbose=0

pretrain_model = lstm_autoencoder

#for ds_name in os.listdir(ds_path):
for ds_name in ['UWaveGestureLibrary']:
    print(f'--- Dataset: {ds_name}')
    print()

    ds_load_result = load_classification_dataset(ds_path, ds_name)
    train_ds, test_ds, preprocessing_layer, label_encoder = ds_load_result

    _x_train, _y_train = train_ds
    x_test, y_test = test_ds

    _x_train = preprocessing_layer(_x_train).numpy()
    x_test = preprocessing_layer(x_test).numpy()

    x_train, x_val, y_train, y_val = train_test_split(
    _x_train, _y_train, test_size=0.2, random_state=42, shuffle=True)

    n_classes = len(label_encoder.classes_)

    print(f'Simple Train >')
    print()

    r = train_evaluate(x_train, y_train, x_val, y_val, x_test, y_test, n_classes, trunk=lstm_trunk, verbose=verbose)

    print_accs(r)

    print(f'Pretrain and Fine Tune >')

    r = pretrain_finetune_evaluate(x_train, y_train, x_val, y_val, x_test, y_test, n_classes, pretrain_model=lstm_autoencoder, verbose=verbose)

    print_accs(r)

    print()
    print()

--- Dataset: UWaveGestureLibrary

Simple Train >


Accuracy | Mean: 0.73 - Std: 0.00 | Max: 0.73
Pretrain and Fine Tune >

Accuracy | Mean: 0.95 - Std: 0.00 | Max: 0.95


